In [1]:
# install pytorch
# ! pip3 install torch torchvision torchaudio

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision  # cnn 계열 도구

In [4]:
# GPU 사용 가능 여부 확인 -> GPU, CPU 선택

if torch.cuda.is_available():   # False
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
# gpu가 가능하다면, gpu를 사용하고, 그렇지 않다면 cpu를 사용한다.
print(device)

cpu


In [ ]:
# 데이터 준비
# 기존 tensorflow 쓸 때 방식의 예시 : tf_keras.datasets.mnist.load_data()
# tensorflow에서는 generator 사용했음 (cnn03 : dogs and cats) > generator로 dataset 구성함
